In [203]:
import speech_recognition as sr # импортируем библиотеки для работы с аудиофайлами
r = sr.Recognizer()
file = sr.AudioFile('train_audio_test_1.wav')
with file as source:
 r.adjust_for_ambient_noise(source)
 audio = r.record(source)
 result = r.recognize_google(audio,language='ru')
print(result)

result2:
{   'alternative': [   {   'confidence': 0.76195019,
                           'transcript': 'начало записи колёсная пара 8760 29 '
                                         'шайба следующий колёсная пара 69 '
                                         'номер 920 6969 29 номер 327 69 327 '
                                         'номер 327 699 29 завод 92 год завод '
                                         '1175 шайба следующих 165'},
                       {   'transcript': 'начало записи колёсная пара 8760 29 '
                                         'шайба следующий колёсная пара 69 '
                                         'номер 920 6969 29 номер 327 69 327 '
                                         'номер 327 699 29 завод 92 год завод '
                                         '1175 шайба следующих 160'},
                       {   'transcript': 'начало записи колёсная пара 8760 29 '
                                         'шайба следующий колёсная пара 69 '
         

In [290]:
# преобразованный текст
result = 'начало записи колесная пара 8760 год 10 завод 29 шайба следующий 69 колесная пара 69 номер 920 69 завод 29 год 83 шайба без буксы следующий колесная пара номер 327 69 327 69 год 92 номер 327699 29 завод шайба следующий колесная пара 32822 год 75 завод 1175 шайба следующий колесная пара 160 691 завод 5 год 10 шайба следующий колесная пара 63369 ошибка номер 63369 год 78 завод 5 гайка следующая запись колесная пара 14038 завод 10 год 10 следующий колесная пара 79859  79859 год 2004 79859 4 год 5 завод пересорт есть что писать 79859 4 год 5 завод а у вас какой пересорт но это же не пересорт но просто они отдельно выписаны они забалансовые это числится либо на забалансе пгк ты сейчас не можешь сказать что они в списке есть то помечай как в спискеследующий колесная пара 13919 а есть 139149 а год завод какой а нет год 90 ошибка 18919 90 год завод 60 следующий колесная пара 13643 завод 5 гайка год 89 '

In [291]:
# сведем ключевые слова к одному виду
result = result.replace('следующих', 'следующий')
result = result.replace('следующая запись', 'следующий')
result = result.replace('следующая', 'следующий')
result = result.replace('начало записи', 'наименование')
result = result.replace('ошибка', "")
result = result.replace('следующий', 'razdel наименование').split('razdel')
result

['наименование колесная пара 8760 год 10 завод 29 шайба ',
 ' наименование 69 колесная пара 69 номер 920 69 завод 29 год 83 шайба без буксы ',
 ' наименование колесная пара номер 327 69 327 69 год 92 номер 327699 29 завод шайба ',
 ' наименование колесная пара 32822 год 75 завод 1175 шайба ',
 ' наименование колесная пара 160 691 завод 5 год 10 шайба ',
 ' наименование колесная пара 63369  номер 63369 год 78 завод 5 гайка ',
 ' наименование колесная пара 14038 завод 10 год 10 ',
 ' наименование колесная пара 79859  79859 год 2004 79859 4 год 5 завод пересорт есть что писать 79859 4 год 5 завод а у вас какой пересорт но это же не пересорт но просто они отдельно выписаны они забалансовые это числится либо на забалансе пгк ты сейчас не можешь сказать что они в списке есть то помечай как в списке',
 ' наименование колесная пара 13919 а есть 139149 а год завод какой а нет год 90  18919 90 год завод 60 ',
 ' наименование колесная пара 13643 завод 5 гайка год 89 ']

In [292]:
# преобразуем записи в список словарей, где ключами являются названия столбцов
list = []
for i in result:
    shedict = {"наименование":[], "номер":[], "год":[], "завод":[], "комментарий":[]}
  
    text = i.split(' ')
    for i in range(len(text)): 
        if text[i] in shedict:
            key = text[i]
            continue
        else:
            shedict[key].append(text[i])
    list.append(shedict)

In [293]:
# почистим данные от лишних слов и цифр
for i in list:
    for key in i:
        i[key] = ' '.join(i[key])      
    
    if not i['номер']:
        i['номер'] = i['наименование']
        i['номер'] = re.sub("[а-я]", "", i['номер'])
    i['номер'] = i['номер'].replace(" ", "")
    i['номер'] = i['номер'][-6:]
    
    i['наименование'] = re.sub("[0-9]", "", i['наименование']) 
    i['комментарий'] = i['год'] + i['завод']
    i['комментарий'] = re.sub("[0-9]", "", i['комментарий']) 
    i['год'] = re.sub("[а-я]", "", i['год']) 
    i['завод'] = re.sub("[а-я]", "", i['завод'])
    
    i['год'] = i['год'].replace(" ", "")
    if i['год']:
        if int(i['год']) > int(2022):
            i['год'] = i['год'][:4]
            if int(i['год']) > int(2022):
                i['год'] = i['год'][:2]
        if int(i['год']) < int(22):
            i['год'] = int(2000) + int(i['год'])
        if int(100) > int(i['год']) > int(22):
            i['год'] = int(1900) + int(i['год'])    

In [298]:
import pandas as pd # импортируем дополнительные библиотеки
import re
df = pd.DataFrame(list) # преобразуем наши данные в таблицу
df.head()

,наименование,номер,год,завод,комментарий
0,колесная пара,8760,2010,29,шайба
1,колесная пара,92069,1983,29,шайба без буксы
2,колесная пара,769929,1992,,шайба
3,колесная пара,32822,1975,1175,шайба
4,колесная пара,160691,2010,5,шайба


In [303]:
# выгружаем полученную таблицу в Excel
#pip install openpyxl

df.to_excel('./git.xlsx')